<a href="https://colab.research.google.com/github/lakshmi-girija-m/Technocolabs-Prerequisite-tasks/blob/main/Toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

     |████████████████████████████████| 788kB 6.0MB/s 
     |████████████████████████████████| 358kB 29.6MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.
     |████████████████████████████████| 122kB 5.6MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.16.1 which is incompatible.


In [8]:
import sys  
!{sys.executable} -m pip install contractions

In [9]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [39]:
pip install scikit-multilearn

     |████████████████████████████████| 92kB 5.2MB/s 


In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re
import contractions
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [11]:
text = pd.read_csv("train.csv", encoding = "ISO-8859-1")

In [34]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
text.loc[[3990,4482,6193,6300,8846,9395,12414,17311], categories]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
3990,0,0,0,0,0,0
4482,0,0,0,0,0,0
6193,0,0,0,0,0,0
6300,0,0,0,0,0,0
8846,0,0,0,0,0,0
9395,0,0,0,0,0,0
12414,0,0,0,0,0,0
17311,0,0,0,0,0,0


In [20]:
def text_preprocess(sent):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer() 
    
    # convert all words to lowercase
    sent = sent.lower()
    #print('Lower: {0} \n'.format(sent))
    
    # expand contractions
    # 48976
    expanded_words = []
    for word in sent.split():
        expanded_words.append(contractions.fix(word))
    sent = ' '.join(expanded_words) 
    #print('Contractions: {0} \n'.format(sent))
    
    # remove html tags
    sent = re.sub('{html}', "", sent)
    #print('HTML: {0} \n'.format(sent))
    
    # remove http links and web site url
    sent = re.sub(r"http\S+", "", sent)
    sent = re.sub(r"www\S+", "", sent)
    #print('Links: {0} \n'.format(sent))
    
    # remove numbers
    sent = re.sub('[0-9]+', '', sent)
    #print('Numbers: {0} \n'.format(sent))
    
    # remove words utc and image
    sent = sent.replace('utc', '')
    #print('utc: {0} \n'.format(sent))
    
    # remove words with .jpg
    sent = re.sub(r"[a-zA-Z]*[0-9]*.jpg", '', sent)
    #print('.jpg: {0} \n'.format(sent))
    
    # remove emails
    sent = re.sub(r"\S*@\S*\s?", '', sent)
    #print('Emails: {0} \n'.format(sent))
    
    # remove single letters
    # sent = re.sub(r"[a-z]{1}", "", sent)
    # print('Single characters: {0} \n'.format(sent))
    
    # tokenization
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sent)
    #print('Tokens: {0} \n'.format(tokens))
    
    # remove stop words
    filtered_words = [w for w in tokens if w not in stopwords.words('english')]
    #print('Stop words: {0} \n'.format(filtered_words))
    
    stem_words = [stemmer.stem(w) for w in filtered_words]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    #print('Lemmatization: {0} \n'.format(lemma_words))
    
    return " ".join(lemma_words)

In [26]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

7000


In [ ]:
for i, sent in enumerate(text.loc[:, "comment_text"]):
    print(i)
    text.loc[i, "comment_text"] = text_preprocess(sent)

In [ ]:
text['comment_text'] = text['comment_text'].map(lambda com : text_preprocess(com))

In [4]:
def remove_long(sent):
  sentence = []
  for word in sent.split():
    if len(word)>20:
      continue
    else:
      sentence.append(word)
  return ' '.join(sentence)

In [ ]:
text['comment_text'] = text['comment_text'].map(lambda com : remove_long(com))

In [35]:
text.to_csv("preprocessed_text.csv", index=False, header=True)

In [81]:
train, test = train_test_split(text, random_state=42, test_size=0.30, shuffle=True)

In [65]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

X_train = train['comment_text']
X_test = test['comment_text']
y_train = train[categories]
y_test = test[categories]

In [82]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('{}: '.format(category))
    LogReg_pipeline.fit(X_train, y_train[category])
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))

toxic: 
Test accuracy is 0.9572610294117647
severe_toxic: 
Test accuracy is 0.9906417112299465
obscene: 
Test accuracy is 0.9776278409090909
threat: 
Test accuracy is 0.9973262032085561
insult: 
Test accuracy is 0.9698153409090909
identity_hate: 
Test accuracy is 0.991915942513369


In [5]:
com = pd.read_csv('preprocessed_text.csv')

In [41]:
com.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [6]:
com.dropna(inplace=True)

In [36]:
Is = pd.DataFrame(com.isnull())

In [7]:
com['comment_text'] = com['comment_text'].map(lambda com : remove_long(com))

In [8]:
X = com['comment_text']

In [ ]:
for i, b in enumerate(Is['comment_text']):
  if b==True:
    print(i)

In [9]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)

tf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True).fit(X_train_counts)
X = tf_transformer.transform(X_train_counts)

In [ ]:
X_data = pd.DataFrame(X.todense())

In [9]:
tf_transformer.idf_

array([ 9.29088213, 10.08938983, 10.58186631, ..., 12.2866144 ,
       12.2866144 , 12.2866144 ])

In [ ]:
vect = TfidfVectorizer(use_idf=True)
tfidf_matrix = vect.fit(X)

In [ ]:
tfidf_matrix = vect.transform(X).todense()

In [ ]:
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
print(df)

In [90]:
train, test = train_test_split(df, random_state=42, test_size=0.30, shuffle=True)

In [92]:
X_train = train[vectorizer.get_feature_names()]
X_test = test[vectorizer.get_feature_names()]
y_train = train[categories]
y_test = test[categories]

KeyError: ignored

In [71]:
clf = LogisticRegression(solver='sag', n_jobs=1)

for category in categories:
    print('{}: '.format(category))
    clf.fit(X_train, y_train[category])
    prediction = clf.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))

toxic: 


ValueError: ignored